In [40]:
from omegaconf import DictConfig, OmegaConf

from symm_rep_learn.models.multivariateCQR import get_coverage, get_set_size
%load_ext autoreload
%autoreload 2

import math
import os
import sys
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.nn import BatchNorm1d, Linear, Sequential
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from omegaconf import OmegaConf

from symm_rep_learn.inference.encp import ENCPConditionalCDF
from symm_rep_learn.inference.ncp import NCPConditionalCDF

cwd = pathlib.Path(os.getcwd())
root_dir = cwd.parent.parent.parent
sys.path.append(str(cwd))
sys.path.append(root_dir)

from paper.experiments.GRF_uncertainty_regression import get_uc_model, plot_gt_and_quantiles, get_proprioceptive_data

# Paths to exemplars form the experiment which we will load.
ncp_path = root_dir / 'paper/results/GRF_regression/kinE_U_go1/robot=go1_lr=0.0001_bs=2048_pat=25_NCP_hu=512_hl=4_act=ELU_resY=False_lstsq=True/gamma=50 seed=1'
encp_path = root_dir / 'paper/results/GRF_regression/kinE_U_go1/robot=go1_lr=0.0001_bs=2048_pat=25_ENCP_hu=512_hl=4_act=ELU_resY=False_lstsq=True/gamma=50 seed=1'
cqr_path = root_dir / 'paper/results/GRF_regression/kinE_U_go1/robot=go1_lr=0.0001_bs=2048_pat=25_CQR_hu=512_hl=4_act=ELU_resY=False_lstsq=True/ seed=1'
paths = {'ncp': ncp_path, 'encp': encp_path, 'cqr': cqr_path}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import escnn
from escnn.nn import FieldType

# Load experiment runs configurations_____________________________________________
configs = {}
for model_name, model_path in paths.items():
    configs[model_name] = OmegaConf.load(model_path/ '.hydra' / 'config.yaml')

# Load dataset______________________________________________________________________
cfg = configs['ncp']   # Any config will do.
cfg.dataset.path = root_dir / cfg.dataset.path
samples, datasets, (rep_x, rep_y), x_moments, y_moments, y_obs_dims = get_proprioceptive_data(configs['ncp'])
(x_train, y_train), (x_val, y_val), (x_test, y_test) = samples
train_ds, val_ds, test_ds = datasets

G = rep_x.group
# lat_rep = G.regular_representation
x_type = FieldType(gspace=escnn.gspaces.no_base_space(G), representations=[rep_x])
y_type = FieldType(gspace=escnn.gspaces.no_base_space(G), representations=[rep_y])

# Load models______________________________________________________________________
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
models = {}
for model_name, model_cfg in configs.items():
    models[model_name] = get_uc_model(model_cfg, x_type, y_type)
    best_path = paths[model_name] / 'last.ckpt'
    state_dict = torch.load(best_path, map_location=device, weights_only=True)['state_dict']
    state_dict = {k.replace('model.', '', 1): v for k, v in state_dict.items()}
    models[model_name].load_state_dict(state_dict)
    models[model_name].to(device).eval()
    print(f"- Loaded trained {model_name}")
    print(f"\tNumber of parameters: {sum(p.numel() for p in models[model_name].parameters())}")

/home/danfoa/Projects/MorphoSymm/morpho_symm/utils/robot_utils.py:152: UserWarning: Representation Q_js found while loading the robot go1. This happens when you load multiple robots in the same session. The old representation will be overwritten.
  warnings.warn(
/home/danfoa/Projects/MorphoSymm/morpho_symm/utils/robot_utils.py:152: UserWarning: Representation TqQ_js found while loading the robot go1. This happens when you load multiple robots in the same session. The old representation will be overwritten.
  warnings.warn(
/home/danfoa/Projects/MorphoSymm/morpho_symm/utils/robot_utils.py:152: UserWarning: Representation E3 found while loading the robot go1. This happens when you load multiple robots in the same session. The old representation will be overwritten.
  warnings.warn(
/home/danfoa/Projects/MorphoSymm/morpho_symm/utils/robot_utils.py:152: UserWarning: Representation R3 found while loading the robot go1. This happens when you load multiple robots in the same session. The old

- qpos_js: C2|[Q_js]:12
- qvel_js: C2|[TqQ_js]:12
- base_lin_acc:base: C2|[R3]:3
- base_lin_vel:base: C2|[R3]:3
- base_lin_vel_err:base: C2|[R3]:3
- base_ang_vel:base: C2|[R3_pseudo]:3
- base_ang_vel_err:base: C2|[R3_pseudo]:3
- gravity_vector:base: C2|[R3]:3
- feet_vel:base: C2|[Rd_on_limbs]:12
- tau_ctrl_setpoint: C2|[TqQ_js]:12
- work: C2|[irrep_0]:1
- kinetic_energy: C2|[irrep_0]:1
- Loaded trained ncp
	Number of parameters: 1677312
- Loaded trained encp
	Number of parameters: 873472
- Loaded trained cqr
	Number of parameters: 1181952


# Configure the models for conditional quantile regression

In [42]:
alpha = cfg.alpha   # All models configured with the same alpha
encp_ccdf = ENCPConditionalCDF(
    model=models['encp'], y_train=y_type(y_train), support_discretization_points=500, lstsq=True
    )
ncp_ccdf = NCPConditionalCDF(
    model=models['ncp'], y_train=y_train, support_discretization_points=500, lstsq=True
    )
# Query the quantiles for all the testing set
q_low_encp, q_high_encp = encp_ccdf.conditional_quantiles(x_cond=x_type(x_test), alpha=alpha)
q_low_ncp, q_high_ncp = ncp_ccdf.conditional_quantiles(x_cond=x_test, alpha=alpha)
q_low_cqr, q_high_cqr = models['cqr'](x_test)  # CQR model is already a quantile regression model.

quantiles = {}
coverage, set_size = {}, {}
for model_name in ['encp', 'ncp', 'cqr']:
    q_low, q_high = locals()[f'q_low_{model_name}'], locals()[f'q_high_{model_name}']
    q_low = torch.tensor(q_low, device=device, dtype=y_train.dtype)
    q_high = torch.tensor(q_high, device=device, dtype=y_train.dtype)
    quantiles[model_name] = (q_low, q_high)
    coverage[model_name] = get_coverage(q_low, q_high, y_test)
    set_size[model_name] = get_set_size(q_low, q_high)
# Print a table with the coverage and set size of the models
pd.DataFrame({'coverage': coverage, 'set_size': set_size})

/tmp/ipykernel_14023/3256168475.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  q_low = torch.tensor(q_low, device=device, dtype=y_train.dtype)
/tmp/ipykernel_14023/3256168475.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  q_high = torch.tensor(q_high, device=device, dtype=y_train.dtype)


,coverage,set_size
encp,tensor(0.6400),tensor(1.9614)
ncp,tensor(0.9172),tensor(1.2942)
cqr,tensor(0.5755),tensor(0.1583)
